In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, column, gridplot

In [7]:
def read_DC_excel(path,sheet):
    """
    import DC cell count data into a pandas dataframe.
    path is absolute path of the DC data, sheet is the sheet number in excel doc.
    return the dataframe
    """
    header_list=[0,1,2]  # which rows to use for multi-index header
    index_col=0
    DC_df=pd.read_excel(path,sheet,header=header_list,index_col=0)
    return DC_df

In [8]:
def find_new_name(old_name):
    """
    For renaming columns in the dataframe, returns '14263' from '14263_IBM' for example
    just manually returning the first 5 characters
    """
    return old_name[0:5]

In [9]:
source_dir=r'C:\Users\danre\Documents\UChicago\SciPy_analysis\Excel_Files'
source_file=r'DC_counts.xlsx'
source_path=os.path.join(source_dir,source_file)
sheet_num=1
DC_df=read_DC_excel(source_path,sheet_num) # read DC counts into a dataframe

In [10]:
old_names=[item[0] for item in DC_df.columns]
DC_df.rename(columns={old_name:find_new_name(old_name) for old_name in old_names},inplace=True)

In [11]:
DC_df_long=DC_df.melt()  # convert to long form
DC_df_long.columns=['Biopsy','Disease','Cell Type','Cell Count']  #rename columns

In [12]:
DC_df_long_dropna=DC_df_long.dropna() 
DC_df_long=DC_df_long_dropna.reset_index().drop(columns='index')

In [13]:
stats=DC_df.describe()
means=stats.loc['mean',:]  #Series object of cell count means for all biopsies and cell types
means=means.to_frame()
mean_counts_groupby_dis=means.groupby(['Disease','Cell Type']).mean() #cell count means over Disease and Type
sem_counts_grouby_dis=means.groupby(['Disease','Cell Type']).sem() #cell count sem's over Disease and Type